# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, make_scorer, fbeta_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from scipy.stats import hmean
from scipy.stats.mstats import gmean
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM Clean_elt_pipeline', con = engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.drop(['id','message','original', 'genre'], axis = 1)
Y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens=word_tokenize(text)
    stopwords = stopwords.tokens('English')
    lemmatizar = WordNetLemmatizer()
    clean_tokens = []
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

In [13]:
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

In [9]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

model = model_pipeline()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred_dataframe = pd.DataFrame(y_pred, columns = Y_test.columns)
y_pred_dataframe.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
for column in Y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(Y_test[column], y_pred_dataframe[column]))

column_name: related

             precision    recall  f1-score   support

          0       0.64      0.34      0.45      1531
          1       0.82      0.94      0.88      4983
          2       0.67      0.40      0.50        40

avg / total       0.78      0.80      0.77      6554

column_name: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5420
          1       0.85      0.38      0.52      1134

avg / total       0.88      0.88      0.86      6554

column_name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

column_name: aid_related

             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3821
          1       0.76      0.53      0.62      2733

avg / total       0.74      0.73      0.72      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# create a function to put all the F1 score for each output. The score should be the median F1 score.

def f1_metric_all(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i], average= 'micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [15]:
def model_pipeline_cv():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'vect__max_df': (0.5, 0.75),
        'clf__estimator__max_depth': [10, 20],
        'vect__ngram_range' : ((1, 1), (1,2)),
        'clf__estimator__n_estimators': [50, 100]
    }
    
    scorer = make_scorer(f1_metric_all,greater_is_better = True)
    
    cv = GridSearchCV(pipeline, param_grid = parameters, scoring= scorer, verbose= 5, n_jobs=1)

    return cv

model_cv = model_pipeline_cv()
model_cv.fit(X_train, Y_train)
y_pred_cv = model_cv.predict(X_test)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.9548367409215746, total=  15.8s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.7s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.9519377479401891, total=  15.7s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   39.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.9562862374122674, total=  15.7s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   58.7s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.9548367409215746, total=  26.8s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.9519377479401891, total=  27.1s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.9562862374122674, total=  26.5s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.9548367409215746, total=  16.3s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.9519377479401891, total=  16.0s
[CV] clf__estimator__max_depth=10, clf__estimato

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 43.6min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred_dataframe_cv = pd.DataFrame(y_pred_cv, columns = Y_test.columns)
for column in Y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(Y_test[column], y_pred_dataframe_cv[column]))

column_name: related

             precision    recall  f1-score   support

          0       1.00      0.00      0.00      1531
          1       0.76      1.00      0.86      4983
          2       0.00      0.00      0.00        40

avg / total       0.81      0.76      0.66      6554

column_name: request

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5420
          1       0.93      0.01      0.02      1134

avg / total       0.85      0.83      0.75      6554

column_name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

column_name: aid_related

             precision    recall  f1-score   support

          0       0.62      0.99      0.76      3821
          1       0.92      0.16      0.27      2733

avg / total       0.75      0.64      0.56      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [8]:
from sklearn.tree import DecisionTreeClassifier
pipeline_svc = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  MultiOutputClassifier(DecisionTreeClassifier()))
    ])

pipeline_svc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer=

In [17]:
def model_pipeline_tr():
    pipeline_svc = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])
    
    parameters_svc = {
        'tfidf__use_idf':[True, False],
        'clf__estimator__max_depth' : [4,8,10]
    }
    
    scorer = make_scorer(f1_metric_all, greater_is_better = True)
    
    cv = GridSearchCV(pipeline_svc, param_grid = parameters_svc, scoring= scorer, verbose= 5)

    return cv

model_tr = model_pipeline_tr()
model_tr.fit(X_train, Y_train)
y_pred_tr = model_tr.predict(X_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__max_depth=4, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=4, tfidf__use_idf=True, score=0.9623893805309734, total=   7.5s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=True ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.1s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=True, score=0.964372902044553, total=   7.7s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=True ................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.5s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=True, score=0.9626182483979249, total=   7.7s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=False ...............


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   24.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=False, score=0.9626182483979249, total=   6.7s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=False ...............


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   32.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=4, tfidf__use_idf=False, score=0.9641440341776015, total=   6.7s
[CV] clf__estimator__max_depth=4, tfidf__use_idf=False ...............
[CV]  clf__estimator__max_depth=4, tfidf__use_idf=False, score=0.9620079340860543, total=   6.8s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=True, score=0.9634574305767469, total=  14.8s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=True, score=0.9646017699115044, total=  15.0s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=True ................
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=True, score=0.9628471162648764, total=  15.0s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=False ...............
[CV]  clf__estimator__max_depth=8, tfidf__use_idf=False, score=0.9641440341776014, total=  13.0s
[CV] clf__estimator__max_depth=8, tfidf__use_idf=False ...........

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  4.1min finished


In [18]:
y_pred_dataframe_tr = pd.DataFrame(y_pred_tr, columns = Y_test.columns)
for column in Y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(Y_test[column], y_pred_dataframe_tr[column]))

column_name: related

             precision    recall  f1-score   support

          0       0.64      0.21      0.32      1577
          1       0.79      0.96      0.87      4930
          2       0.00      0.00      0.00        47

avg / total       0.75      0.78      0.73      6554

column_name: request

             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5416
          1       0.77      0.48      0.59      1138

avg / total       0.88      0.89      0.87      6554

column_name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

column_name: aid_related

             precision    recall  f1-score   support

          0       0.72      0.86      0.78      3884
          1       0.71      0.51      0.59      2670

avg / total       0.72      0.72      0.71      655

From this improve, we can see actually the decision tree algorithm works better than random forest algorithm. I will take this model for my final training model.

### 9. Export your model as a pickle file

In [19]:
import pickle
filename = 'ML_pipeline_model.sav'
pickle.dump(model_tr, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.